**Install pycaret into environment**

In [ ]:
#The code below installs 3.11 (assuming you now have 3.10) and restarts environment, so you can run your cells.
import sys #for version checker
import os #for restart routine

if '3.8' in sys.version:
  print('You already have 3.8, nothing to install')
elif '3.10' in sys.version:
  print("Python version is: ", sys.version)

  print("Printing content of /usr/local/lib/python* to see available versions")
  !ls /usr/local/lib/python*

  #install python 3.8 and dev utils
  #you may not need all the dev libraries, but I haven't tested which aren't necessary.
  !sudo apt-get update -y > /dev/null
  !sudo apt-get install python3.8 python3.8-dev python3.8-distutils libpython3.8-dev > /dev/null
  !sudo apt-get install python3.8-venv binfmt-support  > /dev/null #recommended in install logs of the command above

  #change alternatives
  !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1 > /dev/null
  !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 2 > /dev/null

  # install pip
  !curl -sS https://bootstrap.pypa.io/get-pip.py | python3.8  > /dev/null

  !python3 -m pip install uv

  #install colab's dependencies
  !uv pip install ipython==7.9.0 traitlets==5.7.1 jupyter psutil matplotlib setuptools ipython_genutils ipykernel jupyter_console notebook prompt_toolkit httplib2 astor --system > /dev/null

  #minor cleanup
  !sudo apt autoremove > /dev/null

  #link to the old google package
  !ln -s /usr/local/lib/python3.10/dist-packages/google /usr/local/lib/python3.8/dist-packages/google > /dev/null
  !ln -s /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py /usr/local/lib/python3.8/dist-packages/colab_kernel_launcher.py

  #this is just to verify if 3.11 folder was indeed created
  print("Printing content of /usr/local/lib/python3.8/")
  !ls /usr/local/lib/python3.8/
  !ls /usr/local/lib/python3.8/dist-packages/google

  #restart environment so you don't have to do it manually
  # os.kill(os.getpid(), 9)
else:
  print("Your out of the box Python is not 3.10, so probably the script will not work, so pls feel free to edit the script to ignore then check and re-run: ", sys.version)

You already have 3.8, nothing to install


In [ ]:
!python3 --version

Python 3.8.20


In [ ]:
!pip --version

pip 24.2 from /usr/local/lib/python3.8/dist-packages/pip (python 3.8)


In [ ]:
!pip install uv
!uv pip install pycaret==2.3.5 --system

Audited 1 package in 280ms


**Mount google drive and copy kaggle.json to machine**

In [ ]:
from google.colab import drive
drive.mount('mount')

Drive already mounted at mount; to attempt to forcibly remount, call drive.mount("mount", force_remount=True).


In [ ]:
!cp mount/MyDrive/kaggle.json .
!ls

Bakery.csv  bakery.zip	kaggle.json  mount  sample_data


**Acquire the bakery sales dataset from kaggle and extract zip**


I am using this dataset uploaded 3 years ago on kaggle:

https://www.kaggle.com/datasets/akashdeepkuila/bakery

In [ ]:
!uv pip install kaggle --system

Audited 1 package in 133ms


In [ ]:
!KAGGLE_CONFIG_DIR=$(pwd) kaggle datasets download akashdeepkuila/bakery
!unzip -o bakery.zip

Dataset URL: https://www.kaggle.com/datasets/akashdeepkuila/bakery
License(s): CC0-1.0
bakery.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  bakery.zip
replace Bakery.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

**Load the csv into a pandas dataframe**

In [ ]:
import pandas as pd
data = pd.read_csv('Bakery.csv')
data

,TransactionNo,Items,DateTime,Daypart,DayType
0,1,Bread,2016-10-30 09:58:11,Morning,Weekend
1,2,Scandinavian,2016-10-30 10:05:34,Morning,Weekend
2,2,Scandinavian,2016-10-30 10:05:34,Morning,Weekend
3,3,Hot chocolate,2016-10-30 10:07:57,Morning,Weekend
4,3,Jam,2016-10-30 10:07:57,Morning,Weekend
...,...,...,...,...,...
20502,9682,Coffee,2017-09-04 14:32:58,Afternoon,Weekend
20503,9682,Tea,2017-09-04 14:32:58,Afternoon,Weekend
20504,9683,Coffee,2017-09-04 14:57:06,Afternoon,Weekend
20505,9683,Pastry,2017-09-04 14:57:06,Afternoon,Weekend


**Import pycaret and setup an association experiment**

In [ ]:
from pycaret.arules import *

setup(data, transaction_id='TransactionNo', item_id='Items', session_id = 1337)#, use_gpu=True)

Description,Value
session_id,1337
# Transactions,9465
# Items,94
Ignore Items,None


(       TransactionNo          Items             DateTime    Daypart  DayType
 0                  1          Bread  2016-10-30 09:58:11    Morning  Weekend
 1                  2   Scandinavian  2016-10-30 10:05:34    Morning  Weekend
 2                  2   Scandinavian  2016-10-30 10:05:34    Morning  Weekend
 3                  3  Hot chocolate  2016-10-30 10:07:57    Morning  Weekend
 4                  3            Jam  2016-10-30 10:07:57    Morning  Weekend
 ...              ...            ...                  ...        ...      ...
 20502           9682         Coffee  2017-09-04 14:32:58  Afternoon  Weekend
 20503           9682            Tea  2017-09-04 14:32:58  Afternoon  Weekend
 20504           9683         Coffee  2017-09-04 14:57:06  Afternoon  Weekend
 20505           9683         Pastry  2017-09-04 14:57:06  Afternoon  Weekend
 20506           9684      Smoothies  2017-09-04 15:04:24  Afternoon  Weekend
 
 [20507 rows x 5 columns], 'TransactionNo', 'Items', None, 133

**Train models**

In [ ]:
model = create_model()
model

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cake),(Coffee),0.1039,0.4784,0.0547,0.527,1.1015,0.005,1.1027


In [ ]:
model = create_model(min_support=0.01)
model

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Toast),(Coffee),0.0336,0.4784,0.0237,0.7044,1.4724,0.0076,1.7646
1,(Spanish Brunch),(Coffee),0.0182,0.4784,0.0109,0.5988,1.2518,0.0022,1.3002
2,(Medialuna),(Coffee),0.0618,0.4784,0.0352,0.5692,1.1899,0.0056,1.2109
3,(Pastry),(Coffee),0.0861,0.4784,0.0475,0.5521,1.1542,0.0064,1.1647
4,(Alfajores),(Coffee),0.0363,0.4784,0.0197,0.5407,1.1302,0.0023,1.1356
5,(Juice),(Coffee),0.0386,0.4784,0.0206,0.5342,1.1167,0.0022,1.1199
6,(Sandwich),(Coffee),0.0718,0.4784,0.0382,0.5324,1.1128,0.0039,1.1154
7,(Cake),(Coffee),0.1039,0.4784,0.0547,0.5270,1.1015,0.0050,1.1027
8,(Scone),(Coffee),0.0345,0.4784,0.0181,0.5229,1.0931,0.0015,1.0934
9,(Cookies),(Coffee),0.0544,0.4784,0.0282,0.5184,1.0837,0.0022,1.0832


In [ ]:
model = create_model(min_support=0.01, threshold=0.4)
model

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Toast),(Coffee),0.0336,0.4784,0.0237,0.7044,1.4724,0.0076,1.7646
1,(Spanish Brunch),(Coffee),0.0182,0.4784,0.0109,0.5988,1.2518,0.0022,1.3002
2,(Medialuna),(Coffee),0.0618,0.4784,0.0352,0.5692,1.1899,0.0056,1.2109
3,(Pastry),(Coffee),0.0861,0.4784,0.0475,0.5521,1.1542,0.0064,1.1647
4,(Alfajores),(Coffee),0.0363,0.4784,0.0197,0.5407,1.1302,0.0023,1.1356
5,(Juice),(Coffee),0.0386,0.4784,0.0206,0.5342,1.1167,0.0022,1.1199
6,(Sandwich),(Coffee),0.0718,0.4784,0.0382,0.5324,1.1128,0.0039,1.1154
7,(Cake),(Coffee),0.1039,0.4784,0.0547,0.5270,1.1015,0.0050,1.1027
8,(Scone),(Coffee),0.0345,0.4784,0.0181,0.5229,1.0931,0.0015,1.0934
9,(Cookies),(Coffee),0.0544,0.4784,0.0282,0.5184,1.0837,0.0022,1.0832
